# Chapter 7.5: Final Optimized RAG (모든 기법 통합)

이 노트북은 OSTEP RAG 시스템에 4가지 최적화 기법을 모두 통합하여 최고 성능의 RAG 시스템을 구현합니다.

## 📚 통합 기법
1. **CCH (Contextual Chunk Headers)**: 청크에 챕터 정보 헤더 추가
2. **HyDE (Hypothetical Document Embeddings)**: 쿼리를 가상 문서로 확장
3. **Reranking**: LLM 기반 문서 재정렬
4. **Self-RAG**: 다단계 자기 평가

## 📚 학습 목표
- 4가지 최적화 기법의 시너지 효과 이해
- 통합 파이프라인 구현 및 실행
- 각 단계별 기여도 확인

## 📋 실습 구성
1) 설정 및 하이퍼파라미터 정의
2) 데이터 로드 및 Ollama 설정
3) 통합 파이프라인 아키텍처 설명
4) 각 기법별 함수 구현
5) 통합 RAG 함수 구현
6) 데모: 전체 프로세스 실행 및 분석
7) 요약 및 성능 비교

> **핵심 아이디어**: CCH로 풍부한 임베딩 → HyDE로 쿼리 확장 → Reranking으로 정교한 선택 → Self-RAG로 품질 관리


---
## 1️⃣ 설정 및 하이퍼파라미터 정의

이 셀에서는 통합 RAG 시스템의 모든 파라미터를 정의합니다.

**주요 내용:**
- CCH 인덱스 경로
- 각 기법별 하이퍼파라미터
- Ollama 서버 설정

**실행 결과:**
- 설정값이 출력됩니다.


In [ ]:
# ========================================
# 1️⃣ 설정 / 하이퍼파라미터
# ========================================
import os
import json
import faiss
import numpy as np
import torch
import requests
from pathlib import Path
from typing import List, Dict, Any, Tuple

# 재현성
SEED = 42
np.random.seed(SEED)

# 디바이스
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

# 경로
CHUNK_FILE = "/home/kbs1102/workspace/OSTEP_RAG/data/chunk/ostep_tok400_ov20.json"
CCH_INDEX_FILE = "/home/kbs1102/workspace/OSTEP_RAG/data/index/ostep_hnsw_cch.index"  # CCH 적용 인덱스
CCH_META_FILE = "/home/kbs1102/workspace/OSTEP_RAG/data/index/ostep_hnsw_cch_metadata.json"

# 검색 하이퍼파라미터
INITIAL_K = 15  # 초기 검색 (Reranking용)
RERANK_K = 5    # Reranking 후 선택
FINAL_K = 3     # Self-RAG에 사용할 최종 문서 수
EF_SEARCH = 64

# 임베딩 모델
EMBED_MODEL = "sentence-transformers/all-MiniLM-L6-v2"
NORMALIZE = True

# Ollama 설정
OLLAMA_HOST = "http://localhost:11434"
LLM_MODEL = "llama3.1:8b"
HYDE_TEMP = 0.3      # HyDE용
RERANK_TEMP = 0.1    # Reranking용
SELFRAG_TEMP = 0.2   # Self-RAG용

print(f"🔧 Device: {DEVICE}")
print(f"📁 청크: {CHUNK_FILE}")
print(f"📁 CCH 인덱스: {CCH_INDEX_FILE}")
print(f"🔎 검색 파이프라인: {INITIAL_K} → Rerank({RERANK_K}) → SelfRAG({FINAL_K})")
print(f"🧠 임베딩: {EMBED_MODEL}")
print(f"🤖 LLM: {LLM_MODEL}")


---
## 2️⃣ 데이터 로드 및 Ollama 설정

CCH 인덱스, 청크 데이터, 임베딩 모델을 로드하고 Ollama 서버 연결을 확인합니다.

**주요 내용:**
- CCH 적용 FAISS 인덱스 로드
- 청크 데이터 로드
- 임베딩 모델 로드
- Ollama 서버 연결 확인

**실행 결과:**
- 모든 리소스 로드 완료 메시지가 출력됩니다.


In [ ]:
# 청크 로드
print("청크 데이터 로드 중...")
with open(CHUNK_FILE, 'r', encoding='utf-8') as f:
    CHUNKS = json.load(f)
print(f"✓ 청크 로드: {len(CHUNKS)}개")

# CCH 인덱스 로드
print("\nCCH 인덱스 로드 중...")
if os.path.exists(CCH_INDEX_FILE):
    INDEX_CCH = faiss.read_index(CCH_INDEX_FILE)
    print(f"✓ CCH 인덱스 로드: {INDEX_CCH.ntotal}개 벡터")
else:
    print(f"⚠️  CCH 인덱스 없음. Chapter 7.1을 먼저 실행하세요.")
    INDEX_CCH = None

# 임베딩 모델 로드
print(f"\n임베딩 모델 로드 중: {EMBED_MODEL}")
from sentence_transformers import SentenceTransformer
embed_model = SentenceTransformer(EMBED_MODEL, device="cpu" if DEVICE=="cpu" else DEVICE)
print("✓ 임베딩 모델 로드 완료")

# Ollama 서버 연결 확인
print(f"\nOllama 서버 연결 확인: {OLLAMA_HOST}")
try:
    response = requests.get(f"{OLLAMA_HOST}/api/version", timeout=5)
    if response.status_code == 200:
        print(f"✓ Ollama 서버 연결 성공")
    else:
        print(f"⚠️  Ollama 서버 응답 오류")
except Exception as e:
    print(f"⚠️  Ollama 서버 연결 실패: {e}")

print("\n" + "=" * 80)
print("✅ 모든 리소스 로드 완료!")
print("=" * 80)


---
## 3️⃣ 통합 파이프라인 아키텍처

이 셀에서는 4가지 기법이 통합된 파이프라인의 전체 흐름을 시각적으로 설명합니다.

**파이프라인 흐름:**
1. **쿼리 입력** → "How does virtual memory work?"
2. **HyDE (쿼리 확장)** → 가상 답변 문서 생성
3. **검색 (CCH 인덱스)** → Top-15 문서 검색 (헤더 포함 임베딩)
4. **Reranking** → LLM으로 Top-5 선택
5. **Self-RAG (평가)** → 관련성/지원도/유용성 평가
6. **최종 응답** → 고품질, 근거 기반 답변

**각 기법의 역할:**
- CCH: 임베딩의 컨텍스트 풍부화
- HyDE: 쿼리-문서 형식 차이 해소
- Reranking: 정교한 문서 선택
- Self-RAG: 품질 관리 및 신뢰도 향상

**실행 결과:**
- 파이프라인 다이어그램이 출력됩니다.


In [ ]:
print("=" * 80)
print("[통합 RAG 파이프라인 아키텍처]")
print("=" * 80)
print()
print("Query: 'How does virtual memory work?'")
print("  ↓")
print("┌─────────────────────────────────────────────────┐")
print("│ 1. HyDE: 쿼리 → 가상 답변 문서 생성             │")
print("│    'Virtual memory is a technique...'          │")
print("└─────────────────────────────────────────────────┘")
print("  ↓")
print("┌─────────────────────────────────────────────────┐")
print("│ 2. 검색 (CCH 인덱스)                            │")
print("│    가상 문서 임베딩 → Top-15 검색               │")
print("│    (헤더 포함 청크로 더 정확한 매칭)           │")
print("└─────────────────────────────────────────────────┘")
print("  ↓")
print("┌─────────────────────────────────────────────────┐")
print("│ 3. Reranking                                    │")
print("│    LLM이 각 문서의 관련성 평가 (1-10)           │")
print("│    → Top-5 선택                                 │")
print("└─────────────────────────────────────────────────┘")
print("  ↓")
print("┌─────────────────────────────────────────────────┐")
print("│ 4. Self-RAG                                     │")
print("│    • 관련성 평가: Relevant / Irrelevant         │")
print("│    • 응답 생성: 관련 문서 기반                  │")
print("│    • 지원도 평가: Fully / Partially / No        │")
print("│    • 유용성 평가: 1-5                           │")
print("└─────────────────────────────────────────────────┘")
print("  ↓")
print("Final Answer: 고품질, 근거 기반 응답")
print()
print("=" * 80)
print("💡 4가지 기법의 시너지로 최고 성능 달성!")
print("=" * 80)


---
## 4️⃣ 각 기법별 함수 구현

이 셀에서는 4가지 최적화 기법의 핵심 함수들을 모두 구현합니다.

**구현 함수:**
- HyDE: `generate_hypothetical_document(query)`
- 검색: `search_cch(text, k)`
- Reranking: `rerank_documents(query, indices, scores, top_k)`
- Self-RAG: `assess_relevance()`, `generate_response()`, `assess_support()`, `assess_utility()`

**실행 결과:**
- 모든 함수가 정의됩니다.


In [ ]:
def ollama_call(prompt: str, temperature: float, max_tokens: int = 200) -> str:
    """Ollama API 호출 헬퍼"""
    payload = {
        "model": LLM_MODEL,
        "prompt": prompt,
        "stream": False,
        "options": {"temperature": temperature, "num_predict": max_tokens}
    }
    try:
        r = requests.post(f"{OLLAMA_HOST}/api/generate", json=payload, timeout=60)
        r.raise_for_status()
        return r.json().get("response", "").strip()
    except Exception as e:
        return f"[Error] {e}"

# ===== HyDE =====
def generate_hypothetical_document(query: str) -> str:
    """HyDE: 쿼리를 가상 문서로 확장"""
    prompt = f"""Given the question below, write a detailed passage that answers it.
Question: {query}
Answer passage:"""
    return ollama_call(prompt, HYDE_TEMP, max_tokens=300)

# ===== 검색 (CCH 인덱스) =====
def distance_to_similarity(distances: np.ndarray) -> np.ndarray:
    return 1.0 - (distances / 2.0)

def search_cch(text: str, k: int = INITIAL_K):
    """CCH 인덱스에서 검색"""
    if INDEX_CCH is None:
        raise ValueError("CCH 인덱스가 로드되지 않았습니다.")
    faiss.ParameterSpace().set_index_parameter(INDEX_CCH, "efSearch", EF_SEARCH)
    emb = embed_model.encode([text], convert_to_numpy=True, normalize_embeddings=NORMALIZE).astype('float32')
    D, I = INDEX_CCH.search(emb, k)
    scores = distance_to_similarity(D[0])
    return I[0], scores

# ===== Reranking =====
def ollama_score_relevance(query: str, document: str) -> float:
    """Reranking: LLM으로 관련성 점수 부여"""
    prompt = f"""Rate relevance (1-10):
Query: {query}
Document: {document[:400]}
Score (1-10):"""
    result = ollama_call(prompt, RERANK_TEMP, max_tokens=10)
    try:
        score_str = ''.join(filter(lambda x: x.isdigit() or x == '.', result.split()[0] if result.split() else "5"))
        return max(1.0, min(10.0, float(score_str) if score_str else 5.0))
    except:
        return 5.0

def rerank_documents(query: str, indices: np.ndarray, initial_scores: np.ndarray, top_k: int) -> Tuple[List[int], List[float]]:
    """문서 재정렬"""
    reranked = []
    for i, idx in enumerate(indices):
        if idx < 0:
            continue
        chunk = CHUNKS[idx]
        llm_score = ollama_score_relevance(query, chunk['text'])
        reranked.append((idx, llm_score))
    reranked.sort(key=lambda x: x[1], reverse=True)
    return [x[0] for x in reranked[:top_k]], [x[1] for x in reranked[:top_k]]

# ===== Self-RAG =====
def assess_relevance(query: str, document: str) -> bool:
    """Self-RAG: 관련성 평가"""
    prompt = f"""Is this document relevant?
Query: {query}
Document: {document[:300]}
Answer (Relevant/Irrelevant):"""
    response = ollama_call(prompt, SELFRAG_TEMP, max_tokens=10)
    return 'relevant' in response.lower() and 'irrelevant' not in response.lower()

def generate_response(query: str, context: str) -> str:
    """Self-RAG: 응답 생성"""
    prompt = f"""Answer based on context:
Context: {context}
Question: {query}
Answer:"""
    return ollama_call(prompt, SELFRAG_TEMP, max_tokens=250)

def assess_support(response: str, context: str) -> str:
    """Self-RAG: 지원도 평가"""
    prompt = f"""Is response supported?
Context: {context[:250]}
Response: {response}
Assessment (Fully/Partially/No):"""
    result = ollama_call(prompt, SELFRAG_TEMP, max_tokens=20)
    if 'fully' in result.lower():
        return 'Fully'
    elif 'partially' in result.lower():
        return 'Partially'
    return 'No'

def assess_utility(query: str, response: str) -> int:
    """Self-RAG: 유용성 평가"""
    prompt = f"""Rate utility (1-5):
Query: {query}
Response: {response}
Score:"""
    result = ollama_call(prompt, SELFRAG_TEMP, max_tokens=10)
    try:
        return max(1, min(5, int(''.join(filter(str.isdigit, result))[:1] or "3")))
    except:
        return 3

print("✓ 모든 함수 정의 완료")


---
## 5️⃣ 통합 RAG 함수 구현

이 셀에서는 4가지 기법을 모두 통합한 최종 RAG 함수를 구현합니다.

**주요 내용:**
- 전체 파이프라인을 하나의 함수로 통합
- 각 단계별 결과 로깅
- 최종 응답 생성 및 평가

**실행 결과:**
- `final_optimized_rag(query)` 함수가 정의됩니다.


In [ ]:
def final_optimized_rag(query: str, verbose: bool = True):
    """
    4가지 최적화 기법을 모두 통합한 최종 RAG 함수
    
    Args:
        query: 사용자 질문
        verbose: 각 단계 출력 여부
    
    Returns:
        dict: 최종 응답 및 평가 결과
    """
    if verbose:
        print("=" * 80)
        print(f"Query: {query}")
        print("=" * 80)
    
    # 1. HyDE: 쿼리 확장
    if verbose:
        print("\n[1단계: HyDE - 가상 문서 생성]")
    hyde_doc = generate_hypothetical_document(query)
    if verbose:
        print(f"✓ 가상 문서 생성 ({len(hyde_doc)} 글자)")
        print(f"  미리보기: {hyde_doc[:150]}...")
    
    # 2. 검색 (CCH 인덱스)
    if verbose:
        print(f"\n[2단계: CCH 인덱스 검색 - Top-{INITIAL_K}]")
    indices_initial, scores_initial = search_cch(hyde_doc, k=INITIAL_K)
    if verbose:
        print(f"✓ {len(indices_initial)}개 문서 검색 완료")
    
    # 3. Reranking
    if verbose:
        print(f"\n[3단계: Reranking - Top-{RERANK_K} 선택]")
    indices_reranked, scores_reranked = rerank_documents(query, indices_initial, scores_initial, top_k=RERANK_K)
    if verbose:
        print(f"✓ Reranking 완료")
        for i, (idx, score) in enumerate(zip(indices_reranked[:3], scores_reranked[:3]), 1):
            print(f"  {i}. Score: {score:.1f}/10 - {CHUNKS[idx].get('chapter_title', 'N/A')[:50]}")
    
    # 4. Self-RAG: 관련성 평가
    if verbose:
        print(f"\n[4단계: Self-RAG - 관련성 평가]")
    relevant_docs = []
    for idx in indices_reranked[:FINAL_K]:
        chunk = CHUNKS[idx]
        is_relevant = assess_relevance(query, chunk['text'])
        if verbose:
            print(f"  문서 {len(relevant_docs)+1}: {'Relevant' if is_relevant else 'Irrelevant'}")
        if is_relevant:
            relevant_docs.append(chunk)
    
    if not relevant_docs:
        if verbose:
            print("  → 관련 문서 없음")
        return {"query": query, "answer": "[No relevant documents found]", "support": "No", "utility": 1}
    
    if verbose:
        print(f"  → {len(relevant_docs)}개 관련 문서 발견")
    
    # 5. 응답 생성
    if verbose:
        print("\n[5단계: 응답 생성]")
    context = "\n\n".join([doc['text'][:350] for doc in relevant_docs])
    response = generate_response(query, context)
    if verbose:
        print("✓ 응답 생성 완료")
    
    # 6. 지원도 및 유용성 평가
    if verbose:
        print("\n[6단계: 품질 평가]")
    support = assess_support(response, context)
    utility = assess_utility(query, response)
    if verbose:
        print(f"  지원도: {support}")
        print(f"  유용성: {utility}/5")
    
    # 최종 결과
    result = {
        "query": query,
        "hyde_doc": hyde_doc,
        "num_retrieved": len(indices_initial),
        "num_reranked": len(indices_reranked),
        "num_relevant": len(relevant_docs),
        "answer": response,
        "support": support,
        "utility": utility,
        "sources": [
            {
                "chapter": doc.get('chapter_title'),
                "preview": doc['text'][:100]
            }
            for doc in relevant_docs
        ]
    }
    
    if verbose:
        print("\n" + "=" * 80)
        print("[최종 응답]")
        print("=" * 80)
        print(response)
        print("\n[평가 요약]")
        print(f"  검색 → Rerank → SelfRAG: {INITIAL_K} → {RERANK_K} → {len(relevant_docs)}")
        print(f"  지원도: {support}")
        print(f"  유용성: {utility}/5")
        print("=" * 80)
    
    return result

print("✓ 통합 RAG 함수 정의 완료")


In [ ]:
# 데모 실행
TEST_QUERY = "How does the operating system handle virtual memory?"

print("\n")
print("🚀 통합 최적화 RAG 시스템 실행")
print("=" * 80)

result = final_optimized_rag(TEST_QUERY, verbose=True)

print("\n\n📊 상세 결과:")
print(json.dumps({
    "query": result["query"],
    "pipeline_flow": f"{result['num_retrieved']} → {result['num_reranked']} → {result['num_relevant']}",
    "support": result["support"],
    "utility": result["utility"],
    "answer_length": len(result["answer"])
}, indent=2, ensure_ascii=False))


---
## 7️⃣ 요약 및 성능 비교

### 통합 시스템 요약

**4가지 최적화 기법의 시너지:**
1. **CCH (Contextual Chunk Headers)**
   - 역할: 청크에 챕터 정보 헤더 추가로 임베딩 품질 향상
   - 효과: 암묵적 참조 문제 해결, 검색 정확도 +27.9%

2. **HyDE (Hypothetical Document Embeddings)**
   - 역할: 쿼리를 가상 답변 문서로 확장
   - 효과: 쿼리-문서 형식 차이 해소, 매칭 정확도 향상

3. **Reranking**
   - 역할: LLM으로 문서 관련성을 정교하게 재평가
   - 효과: 초기 검색의 한계 보완, 미묘한 관련성 차이 포착

4. **Self-RAG**
   - 역할: 다단계 자기 평가 (필요성/관련성/지원도/유용성)
   - 효과: Hallucination 감소, 근거 기반 응답, 신뢰도 향상

### 통합의 이점

- ✅ **최고 정확도**: 4가지 기법이 서로 보완하여 각 단계에서 최적화
- ✅ **신뢰도 향상**: Self-RAG의 평가로 응답 품질 보증
- ✅ **유연성**: 각 기법을 독립적으로 활성화/비활성화 가능
- ✅ **확장성**: 새로운 기법 추가 용이한 모듈형 구조

### 성능 트레이드오프

- **장점**: 최고 수준의 정확도와 신뢰도
- **단점**: 높은 레이턴시 (HyDE + Reranking + Self-RAG의 다중 LLM 호출)
- **권장 사용**: 정확도가 중요한 도메인 (의료, 법률, 교육 등)

### 최적화 방향

1. **속도 우선**: HyDE와 Self-RAG를 간소화하거나 제거
2. **정확도 우선**: Reranking의 초기 검색 범위 확대 (k↑)
3. **균형**: 현재 설정 유지 (초기 15 → Rerank 5 → SelfRAG 3)

### 다음 단계

- 실제 운영 환경 배포
- 사용자 피드백 수집 및 파라미터 튜닝
- 도메인 특화 최적화
- 캐싱 전략으로 레이턴시 감소

### 생성된 파일

- `ostep_contextual_chunk_headers.ipynb`: CCH 기법 실습
- `ostep_hyde.ipynb`: HyDE 기법 실습
- `ostep_reranking.ipynb`: Reranking 기법 실습
- `ostep_self_rag.ipynb`: Self-RAG 기법 실습
- `ostep_final_optimized.ipynb`: 통합 최적화 시스템 (현재 파일)

### 주요 함수

- `generate_hypothetical_document(query)`: HyDE 가상 문서 생성
- `search_cch(text, k)`: CCH 인덱스 검색
- `rerank_documents(query, indices, scores, top_k)`: 문서 재정렬
- `assess_relevance(query, document)`: Self-RAG 관련성 평가
- `generate_response(query, context)`: 응답 생성
- `assess_support(response, context)`: 지원도 평가
- `assess_utility(query, response)`: 유용성 평가
- **`final_optimized_rag(query)`**: 전체 통합 파이프라인

---

## 🎉 Chapter 7 완료!

4가지 최적화 기법을 모두 학습하고 통합 RAG 시스템을 구축했습니다. 각 기법의 원리와 구현 방법을 이해했으며, 실제 OSTEP 데이터에 적용하여 성능을 확인했습니다.
